In [1]:
import os
from pymongo import MongoClient
import pandas as pd
import json
from googleapiclient.discovery import build

#### Importing videos csv

In [ ]:
videos_df = pd.read_csv('./csvs/modified_videos_xx.csv')     # last updated videos dataset

In [ ]:
videos_df.info()

#### Getting list of youtube_id for videos missing category_id in live dataset 

In [ ]:
videos_YTids_noCategoryIds_list = list(videos_df.loc[videos_df['category_id'].isna(),'youtube_id'])
videos_YTids_noCategoryIds_list[:5]

### YouTube API key

In [ ]:
developer_key = os.environ.get('DEVELOPER_KEY')

youtube = build('youtube', 'v3', developerKey=developer_key) # project-0127 Data Cleaning

#### Fetching category id of videos

In [ ]:
part_string = 'snippet'
videos_youtube_id = videos_YTids_noCategoryIds_list

In [ ]:
videos_categoryId_dict = {}
for id in range(0,len(videos_youtube_id)):
    print("id: ", id)
    
    try:
        request = youtube.videos().list(
            part=part_string,
            id=videos_youtube_id[id],
        )
        response = request.execute()
        
        if response['items']!=[] and response['items'][0]['snippet']['categoryId'] != '':

            videos_categoryId = response['items'][0]['snippet']['categoryId']
        
        else:
            videos_categoryId = None

    except Exception as e:
        print("An exception occured")
        print(e)
        videos_categoryId = None

    videos_categoryId_dict[videos_youtube_id[id]] = videos_categoryId

In [ ]:
category_id = open('./youtube_api/videos_category_id.txt',"a")
category_id.write(json.dumps(videos_categoryId_dict))
category_id.close()

### Connection to MongoDB

In [3]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to videos collection

In [ ]:
videos = database['modified_videos_xx']     # updated videos collection

#### Updating video category id

In [ ]:
try:
    for (key,value) in videos_categoryId_dict.items():
    videos.update_many(
    {"youtube_id" : key},
    {"$set": { "category_id" : value}}
    );
except Exception as e:
    print(exception)